<a href="https://colab.research.google.com/github/VectorInstitute/Causal_Inference_Laboratory/blob/main/notebooks/Hands_On_Session1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Welocme to Hands-on Session #1

---

**Problem:** To estimate the causal impact of job training on unemployment

---

**Jobs Dataset**

The Jobs by LaLonde (1986) is a widely used benchmark in the causal inference community, where the treatment is job training and the outcomes are binary outcomes of unemployment. This dataset combines a randomized study based on the National Supported Work program with observational data to form a larger dataset (Smith&Todd, 2005). The presence of the randomized subgroup gives a way to estimate the “groundtruth” causal effect. 


---
**This notebook**
Our dataset is the preprocessed jobs dataset (also available online https://www.fredjo.com/): it contains the LaLonde experimental sample (297 treated and 425 control) and the PSID comparison group (2490 control). The Jobs dataset is already split into the train/test (2570/642) splits in a 80/20 split.
- X: There are 17 covariates such as age and education, as well as previous earnings.
- T: Treatment
- Y: Binary factual outcomes on unemployment
---

Let's start!

In [ ]:
!git clone https://github.com/VectorInstitute/Causal_Inference_Laboratory.git
!mv Causal_Inference_Laboratory code
!mv code/data ./data
!mv code/utils ./utils
!mv code/models ./models
!mv code/estimation_results ./estimation_results

Cloning into 'Causal_Inference_Laboratory'...
remote: Enumerating objects: 328, done.
remote: Counting objects: 100% (328/328), done.
remote: Compressing objects: 100% (272/272), done.
remote: Total 328 (delta 135), reused 212 (delta 51), pack-reused 0
Receiving objects: 100% (328/328), 23.88 MiB | 10.92 MiB/s, done.
Resolving deltas: 100% (135/135), done.
Updating files: 100% (98/98), done.


## Imports

In [ ]:
import os
import tensorflow as tf
import numpy as np
import pandas as pd
import time
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, LogisticRegression, Ridge
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor
from typing import Dict

import utils.estimators as models
import utils.preprocessing as helper
from utils.preprocessing import sys_config
import utils.metrics as metrics
from utils.evaluation import *

import warnings
warnings.filterwarnings('ignore')

In [ ]:
datasets_folder = sys_config["datasets_folder"]
results_folder = sys_config["results_folder"]

seed = 0
np.random.seed(seed)

**Data Loading**

In this module, we provide the data loading for the Jobs dataset. We will only use one realization of it.

In [ ]:
# Let's load the data
dataset_name = "Jobs"

# Load covariates, treatment, and factual outcomes for the training and test datasets
x_train_all, t_train_all, yf_train_all = helper.load_Jobs_observational(datasets_folder, dataset_name, details=True)
x_test_all, t_test_all, yf_test_all = helper.load_Jobs_out_of_sample(datasets_folder, dataset_name, details=True)

-------------------------------------------------------------------------------
The details of the train split of Jobs dataset:
Number of realizations: 10
ate (1, 1) float64
e (2570, 10) float64
I (2570, 10) int32
yadd (1, 1) uint8
yf (2570, 10) float64
t (2570, 10) float64
x (2570, 17, 10) float64
ymul (1, 1) uint8
-------------------------------------------------------------------------------
-------------------------------------------------------------------------------
The details of the test split of Jobs dataset:
Number of realizations: 10
ate (1, 1) float64
e (642, 10) float64
I (642, 10) int32
yadd (1, 1) uint8
yf (642, 10) float64
t (642, 10) float64
x (642, 17, 10) float64
ymul (1, 1) uint8
-------------------------------------------------------------------------------


In [ ]:
# We will only use the first realization
realization = 0
x_train, t_train, yf_train = x_train_all[:, :, realization], t_train_all[:, realization], yf_train_all[:, realization]
x_test, t_test, yf_test = x_test_all[:, :, realization], t_test_all[:, realization], yf_test_all[:, realization]

print(x_train.shape, t_train.shape, yf_train.shape)
print(x_test.shape, t_test.shape, yf_test.shape)

t_value_train, count_t_train = np.unique(t_train, return_counts=True)
yf_value_train, count_yf_train = np.unique(yf_train, return_counts=True)

print(f"Number of T = {int(t_value_train[0])} is {count_t_train[0]} and \
number of T = {int(t_value_train[1])} is {count_t_train[1]}.")
print(f"Number of Y = {int(yf_value_train[0])} is {count_yf_train[0]} and \
number of Y = {int(yf_value_train[1])} is {count_yf_train[1]}.")

(2570, 17) (2570,) (2570,)
(642, 17) (642,) (642,)
Number of T = 0 is 2333 and number of T = 1 is 237.
Number of Y = 0 is 378 and number of Y = 1 is 2192.


In the Jobs dataset, though there is no ground-truth CATE as in IHDP, which is a synthetic dataset, we have ground-truth ATE. 

In [ ]:
# Load Ground Truth facutal and counterfactual outcomes for the training and test datasets
ate_in_gt, ate_out_gt = helper.load_Jobs_ground_truth(datasets_folder, dataset_name, details=False)
ate_in_gt, ate_out_gt = ate_in_gt.item(), ate_out_gt.item()
print(ate_in_gt, ate_out_gt)

0.07794018617548037 0.07794018617548037


## Estimation
Now you can build causal estimators to measure the impact of training on unemployment given all the available $(X, T, YF)$ tuples. 

A few reminders:
- we have already done the train-test split (80/20 split) for you; for further splitting, a train/validation/test split with ratios 56/24/20 is used in the TAR-Net paper.
- the outcomes are binary values.
- we have the ground-truth ATE.


In [ ]:
# Given 
# x_train, t_train, yf_train 
# x_test, t_test, yf_test
# ate_in_gt, ate_out_gt

## Q1: S-learner
In this part, you are asked to build any S-learner (viewing t as one feature).

In [ ]:
def train_and_evaluate_slearner(x, t, yf, x_t):
    """
    Training a s-leaner
    :param x: covariates
    :param t: treatment
    :param yf: factual outcomes
    :param x_t: out-of-sample covariates
    :return:
    """    
    ####################
    #PUT YOUR CODE HERE#
    ####################
    return ate_in, ate_out



## Q2: T-learner
In this part, you are asked to build any T-learner (building separate models for different treatment groups).

In [ ]:
def train_and_evaluate_tlearner(x, t, yf, x_t):
    """
    Training a t-leaner
    :param x: covariates
    :param t: treatment
    :param yf: factual outcomes
    :param x_t: out-of-sample covariates
    :return:
    """    
    ####################
    #PUT YOUR CODE HERE#
    ####################
    return ate_in, ate_out

## Q3: Deep Learning-based learner
In this part, you are asked to build any estimator like TAR-Net/Dragonnet.

In [ ]:
def train_and_evaluate_dllearner(x, t, yf, x_t):
    """
    Training a deep learning-based leaner
    :param x: covariates
    :param t: treatment
    :param yf: factual outcomes
    :param x_t: out-of-sample covariates
    :return:
    """    
    ####################
    #PUT YOUR CODE HERE#
    ####################
    return ate_in, ate_out

## Q4: IPW
In this part, you are asked to build an estimator based on Inverse Propensity Weighting.

In [ ]:
def train_and_evaluate_ipw(x, t, yf, x_t):
    """
    Training a deep learning-based leaner
    :param x: covariates
    :param t: treatment
    :param yf: factual outcomes
    :param x_t: out-of-sample covariates
    :return:
    """    
    ####################
    #PUT YOUR CODE HERE#
    ####################
    return ate_in, ate_out

## Q5: Double Machine Learning
In this part, you are asked to build any R-learner (Double machine learning estimator).

In [ ]:
def train_and_evaluate_dml(x, t, yf, x_t):
    """
    Training a double machine learning leaner
    :param x: covariates
    :param t: treatment
    :param yf: factual outcomes
    :param x_t: out-of-sample covariates
    :return:
    """    
    ####################
    #PUT YOUR CODE HERE#
    ####################
    return ate_in, ate_out

## Evaluation


In [ ]:
def calculate_mae_ATE(ate, ate_gt):
    """
    Calculate the absolute error of ATE estimation
    :param ate: predicted ate
    :param ate_gt: ground-truth ate
    :return:
    """
    return np.abs(ate - ate_gt)

estimator_set = ["S-Learner", "T-Learner", "DL-Leaner", "IPW", "DML"]

for estimator in estimator_set:
    if estimator == "S-Learner":
        ate_in, ate_out = train_and_evaluate_slearner(x_train, t_train, yf_train, x_test)
    elif estimator == "T-Learner":
        ate_in, ate_out = train_and_evaluate_tlearner(x_train, t_train, yf_train, x_test)
    elif estimator == "DL-Learner":
        ate_in, ate_out = train_and_evaluate_dllearner(x_train, t_train, yf_train, x_test)
    elif estimator == "IPW":
        ate_in, ate_out = train_and_evaluate_slearner(x_train, t_train, yf_train, x_test)
    elif estimator == "DML":
        ate_in, ate_out = train_and_evaluate_slearner(x_train, t_train, yf_train, x_test)
    else:
        print("Undefined estimator")
        ate_in, ate_out = None, None

    text = f" Estimation via {estimator}"
    print(f"{text:-^79}")
    # in-sample
    print(f"Absolute error of in-sample ATE of {estimator}: {calculate_mae_ATE(ate_in, ate_in_gt) if ate_in is not None else 'N/A'}")

    # out-of-sample
    print(f"Absolute error of out-of-sample ATE of {estimator}: {calculate_mae_ATE(ate_out, ate_out_gt) if ate_in is not None else 'N/A'}")

## Reflection